In [1]:
import pymysql as db
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

DB 연결 및 SQL문 확인

In [2]:
# user = 'jyji'
# password = '123123'
# host = 'localhost'
# dbname = 'IMDB'
# charset = 'utf8'
con = db.connect(host='localhost', user='jyji', password='123123', db='IMDB', charset='utf8')
cur = con.cursor()
sql = 'select * from actors where id=20000;'
cur.execute(sql)
rows = cur.fetchall()
print(rows)
con.close()

((20000, 'Eric', 'Aude', 'M'),)


In [3]:
def get_connection():
    con = db.connect(host='localhost', user='jyji', password='123123', db='IMDB', charset='utf8')
    return con

In [4]:
with get_connection() as con:
    SQL = open('./SQL/movie_related_information.sql').read()
    df = pd.read_sql(SQL, con)
    print(df.head())
    df.to_csv('./data/movie_related_information.csv')

   MID                  Actor Movie              Role  Year
0    2  Darrell (I) Armstrong     $  Associated Press  1971
1    2          Warren Beatty     $       Joe Collins  1971
2    2        Scott (I) Brady     $             Sarge  1971
3    2          Arthur Brauss     $         Candy Man  1971
4    2             Gert Fröbe     $        Mr. Kessel  1971


In [5]:
with get_connection() as con:
    SQL = open('./SQL/director_related_information.sql').read()
    df = pd.read_sql(SQL, con)
    print(df.head())
    df.to_csv('./data/director_related_information.csv')

     MID        Director                           Movie  Year
0   1114  A. Aleksandrov                     14/1 endlos  1998
1  17209        A. Cyran                   Animali pazzi  1939
2  36457    A. Ignatenko  Bianco, il giallo, il nero, Il  1975
3  36457    A. Ignatenko  Bianco, il giallo, il nero, Il  1975
4  36457    A. Ignatenko  Bianco, il giallo, il nero, Il  1975


Pandas Dataframe으로 변환

In [6]:
movie_df = pd.read_csv('./data/movie_related_information.csv', index_col=[0])
director_df = pd.read_csv('./data/director_related_information.csv', index_col=[0])

In [7]:
movie_df.head()

,MID,Actor,Movie,Role,Year
0,2,Darrell (I) Armstrong,$,Associated Press,1971
1,2,Warren Beatty,$,Joe Collins,1971
2,2,Scott (I) Brady,$,Sarge,1971
3,2,Arthur Brauss,$,Candy Man,1971
4,2,Gert Fröbe,$,Mr. Kessel,1971


1세 2세 (I) (II) 와 같은 regnal number 뒤로 위치 바꾸기

In [8]:
def change_regnal_number(x):
    l = x.split(' ')
    if l[1].startswith('('):
        l[1], l[2] = l[2], l[1]
    return ' '.join(l)


movie_df['Actor'] = movie_df['Actor'].apply(change_regnal_number)
movie_df.head()

,MID,Actor,Movie,Role,Year
0,2,Darrell Armstrong (I),$,Associated Press,1971
1,2,Warren Beatty,$,Joe Collins,1971
2,2,Scott Brady (I),$,Sarge,1971
3,2,Arthur Brauss,$,Candy Man,1971
4,2,Gert Fröbe,$,Mr. Kessel,1971


In [9]:
director_df.sample(10)

,MID,Director,Movie,Year
20222,14141,Wellson Chin,"American President, The",1995
15069,63635,Paul (I) Powell,Ch Forte com Limo,1993
12076,28414,Lorenzo Gicca Palli,Balkanski spijun,1981
2659,72473,Byron Shah,Cracking Up,1983
8752,77076,Jamil Sulong,Dalalf,1984
17309,59862,Ryunosuke Otonashi,Chasing Cain,2001
19534,80594,Toshiro Tsuchida,Dead Room,2001
5518,83396,Ernst R. von Theumer,Des terroristes la retraite,1985
7333,77612,György Szomjas,Dance Hall,1941
20767,13875,Yin-jung Chen,American Dreamer,1984


In [10]:
director_df['Director'] = director_df['Director'].apply(change_regnal_number)
director_df.sample(10)

,MID,Director,Movie,Year
9626,86819,Joel D. Wynkoop,Dimenticare Palermo,1990
17965,39864,Shuji Kataoka,"Black Scorpion, The",1957
19131,9976,Thor L. Brooks,Alaska Seas,1954
3424,79510,Chung Ting (II),David (1979/I),1979
19770,79634,Valeri Todorovsky,"Davy Crockett, Indian Scout",1950
11029,51442,Katsuya Matsumura,"C't' ton tour, Laura Cadieux",1998
13283,88766,Mehmet Özcelik,Doc Hollywood,1991
7630,81099,Hector Ugalde,Death Force,1978
5637,40893,Evgen Khovostyank,Block-Heads,1938
17545,40800,Sanjay Khan,Blink,1994


In [11]:
movie_df.head()

,MID,Actor,Movie,Role,Year
0,2,Darrell Armstrong (I),$,Associated Press,1971
1,2,Warren Beatty,$,Joe Collins,1971
2,2,Scott Brady (I),$,Sarge,1971
3,2,Arthur Brauss,$,Candy Man,1971
4,2,Gert Fröbe,$,Mr. Kessel,1971


두 테이블을 조인해서 다시 저장

In [12]:
joined_df = pd.merge(left=movie_df, right=director_df, how='inner', on=['MID', 'Movie', 'Year'])
joined_df = joined_df.drop('MID', axis=1)
joined_df = joined_df[['Actor', 'Director', 'Movie', 'Role', 'Year']]
joined_df.head(10)

,Actor,Director,Movie,Role,Year
0,Darrell Armstrong (I),Les 12 Poissons,$,Associated Press,1971
1,Warren Beatty,Les 12 Poissons,$,Joe Collins,1971
2,Scott Brady (I),Les 12 Poissons,$,Sarge,1971
3,Arthur Brauss,Les 12 Poissons,$,Candy Man,1971
4,Gert Fröbe,Les 12 Poissons,$,Mr. Kessel,1971
5,Bob Herron,Les 12 Poissons,$,Bodyguard,1971
6,Horst Hesslein,Les 12 Poissons,$,Bruno,1971
7,Hans Hutter (I),Les 12 Poissons,$,Karl,1971
8,Wolfgang Kieling,Les 12 Poissons,$,Granich,1971
9,Wolfgang Kuhlman,Les 12 Poissons,$,Furcoat,1971


In [13]:
movie_df.to_csv('./data/movie_related_information.csv')
director_df.to_csv('./data/director_related_information.csv')
joined_df.to_csv('./data/joined_table.csv')

In [14]:
movie_df = pd.read_csv('./data/movie_related_information.csv', index_col=[0])
director_df = pd.read_csv('./data/director_related_information.csv', index_col=[0])
joined_df = pd.read_csv('./data/joined_table.csv', index_col=[0])

단어 토큰 준비

In [16]:
joined_df['Actor']

0         Darrell Armstrong (I)
1                 Warren Beatty
2               Scott Brady (I)
3                 Arthur Brauss
4                    Gert Fröbe
                  ...          
800109            Kevin Wiggins
800110           Bronwyn Andrus
800111       Christine McPeters
800112         Jeanette Wiggins
800113              Paige Witte
Name: Actor, Length: 800114, dtype: object

In [4]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}


for col in joined_df.columns:
    word_list = joined_df[col].tolist()
    word_list = list(map(str, word_list))
    word_list = list(set(word_list))
    len_word_dict = len(word_dict)
    for i, w in enumerate(word_list):
        word_dict[w] = i + len_word_dict

number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(number_dict)

len(word_dict), len(number_dict), vocab_size

(202880, 202880, 202880)

In [29]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
word_dict_col = {}
col_list = joined_df.columns.tolist()

for col in joined_df.columns:
    word_dict_col[col] = dict(word_dict)
    word_list = joined_df[col].tolist()
    word_list = list(map(str, word_list))
    word_list = list(set(word_list))
    len_word_dict = len(word_dict)
    for i, w in enumerate(word_list):
        word_dict_col[col][w] = i + len_word_dict

number_dict_col = {c: {i: w for i, w in enumerate(word_dict_col[c])} for c in joined_df.columns }

In [34]:
import random

random.sample(list(word_dict_col['Actor']), 10), random.sample(list(number_dict_col['Actor']), 10)

(['Casey Kern',
  'Rocky Marciano',
  'Bob Dorian',
  'Philippe Brigaud',
  'Wolfgang Hillinger',
  'Janette Bertrand',
  'Inez Griffin',
  'Danièle Gain',
  'Sue Mitobe',
  'Lucie Mannheim'],
 [18913, 51206, 16882, 5814, 57471, 51731, 57223, 83375, 74580, 15609])

joined dataframe을 복사하여 토큰 테이블로 바꾸기

In [36]:
token_df = joined_df.copy()

In [37]:
token_df.head()

,Actor,Director,Movie,Role,Year
0,Darrell Armstrong (I),Les 12 Poissons,$,Associated Press,1971
1,Warren Beatty,Les 12 Poissons,$,Joe Collins,1971
2,Scott Brady (I),Les 12 Poissons,$,Sarge,1971
3,Arthur Brauss,Les 12 Poissons,$,Candy Man,1971
4,Gert Fröbe,Les 12 Poissons,$,Mr. Kessel,1971


In [38]:
for col in token_df.columns:
    token_df[col] = token_df[col].apply(lambda x: str(x))

In [39]:
token_df.head()

,Actor,Director,Movie,Role,Year
0,Darrell Armstrong (I),Les 12 Poissons,$,Associated Press,1971
1,Warren Beatty,Les 12 Poissons,$,Joe Collins,1971
2,Scott Brady (I),Les 12 Poissons,$,Sarge,1971
3,Arthur Brauss,Les 12 Poissons,$,Candy Man,1971
4,Gert Fröbe,Les 12 Poissons,$,Mr. Kessel,1971


In [40]:
type(token_df['Year'][0])

str

In [41]:
def change_to_token(entity, col):
    return word_dict_col[col][entity]

for col in token_df.columns:
    token_df[col] = token_df[col].apply(change_to_token, args=(col,))

token_df.head()

,Actor,Director,Movie,Role,Year
0,35751,6368,6913,73199,12
1,11705,6368,6913,53603,12
2,66357,6368,6913,42415,12
3,73965,6368,6913,287,12
4,41111,6368,6913,57800,12


In [46]:
for i, entity in enumerate(token_df.iloc[0, :].values):
    print(number_dict_col[col_list[i]][entity], end=', ')

Darrell Armstrong (I), Les 12 Poissons, $, Associated Press, 1971, 

토큰 테이블 저장

In [47]:
token_df.to_csv('./data/token_table.csv')

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}